In [49]:
import boto3
import re,os
import pandas as pd
import h5py

with open(r"Z:\LWI\LWI S3 Key.txt") as secret_file:
    secret = secret_file.readlines()
secret_ID = secret[1].strip('\n')
secret_key = secret[3].strip('\n')

In [3]:
client = boto3.client('s3',
                   aws_access_key_id=secret_ID,
                   aws_secret_access_key = secret_key)

session = boto3.Session(aws_access_key_id=secret_ID,
                        aws_secret_access_key = secret_key)

s3 = session.resource('s3')

In [14]:
response = client.list_buckets()
bucket_names = [bucket['Name'] for bucket in response['Buckets']]
# Limiting to buckets I have access to.
bucket_names = [
 'lwi-region4',
 'lwi-region5',
 'lwi-region6',
 'lwi-region7',
 ]


In [41]:
# response = client.list_buckets()
# create a dictionary of hms and ras models by bucket name
models = {}
for bucket in bucket_names:
    print('\nSearching Bucket:', bucket)
    my_bucket = s3.Bucket(bucket)
    models[bucket] = {
        'hms': [],
        'ras': {
            'all_hdfs': [],
            'plan_hdfs': [],
            # 'models': {
            #     'directory': [],
            #     'plans': {
            #         'file': '',
            #         'name': '',
            #         'description': '',
            #         'ras_type': '',
            #     },
            # },
        },
    }
    for obj in my_bucket.objects.all():
        
        if obj.key.endswith('hms'):
            models[bucket]['hms'].append(obj.key)
        
        if obj.key.endswith('hdf'):
            # If 'HMS' in obj.key dont add to ras['all_hdfs']
            if 'hms' not in obj.key.lower():
                models[bucket]['ras']['all_hdfs'].append(obj.key)
                # If .p**.hdf add to ras['plan_hdfs']
                if re.search('p...hdf', obj.key):   
                        # print(obj.key.split('/')[-1])
                        models[bucket]['ras']['plan_hdfs'].append(obj.key)
            
models



Searching Bucket: lwi-region4

Searching Bucket: lwi-region5

Searching Bucket: lwi-region6

Searching Bucket: lwi-region7


{'lwi-region4': {'hms': ['deliverables/20221006_12010005_SL_HMSModelSetup/02_Simulation/HEC-HMS/Lower_Sabine.hms',
   'deliverables/20221021_08080204_WH_HMSModelSetup/00_Model/WhiskyChitto_10.19.22/WhiskyChitto.hms',
   'deliverables/20221215_12010004_TO_HMSModelSetup/01_Models/HEC_HMS/TO_Detailed/TO_Detailed.hms',
   'deliverables/20221215_12010004_TO_HMSModelSetup/01_Models/HEC_HMS/TO_LimitedDetail/TO_LimitedDetail.hms',
   'deliverables/20221222_08080203_UC_HMSModelSetup/00_Model/UpperCalcasieu/UpperCalcasieu.hms',
   'deliverables/20221222_08080205_WF_HMSModelSetup/00_Model/WestForkCalcasieu/WestForkCalcasieu.hms'],
  'ras': {'all_hdfs': ['deliverables/20221222_08080203_UC_RAS2DModelSetupP1/UC_UpperCalcasieu/03_Terrain/UC_Terrain.hdf',
    'deliverables/20221222_08080203_UC_RAS2DModelSetupP1/UC_UpperCalcasieu/04_MapLayers/NLCD_Manning_Roughness.backup.hdf',
    'deliverables/20221222_08080203_UC_RAS2DModelSetupP1/UC_UpperCalcasieu/04_MapLayers/NLCD_Manning_Roughness.hdf',
    'deli

In [25]:

models_df = pd.DataFrame(models)
# models_df = models_df.transpose()
models_df

# pd.json_normalize(models).transpose()

,lwi-region4,lwi-region5,lwi-region6,lwi-region7
hms,[deliverables/20221006_12010005_SL_HMSModelSet...,[],[],[deliverables/Amite_HMS_Backcheck_/HMS/AM_Amit...
ras,{'all_hdfs': ['deliverables/20221222_08080203_...,{'all_hdfs': ['deliverables/2023_0127_ToDOTD_H...,{'all_hdfs': ['deliverables/20230106_TO3_Hydro...,{'all_hdfs': ['deliverables/Amite_RAS_Setupdel...


In [79]:
import os
model_dirs = {}
for region in models:
    print(region)
    model_dirs[region] = {
       'dirs' : []
    }
    # For each plan hdf get the model name, description, and ras type (1D, 2D, etc.), and pull a geospatial extent from the hdf.
    # get  model directories
    for plan in models[region]['ras']['plan_hdfs']:
        # print(plan)
        head, tail = os.path.split(plan)
        # print(head)
        model_dirs[region]['dirs'].append(head)

model_dirs['lwi-region4']['dirs']



lwi-region4
lwi-region5
lwi-region6
lwi-region7


['deliverables/20221222_08080203_UC_RAS2DModelSetupP1/UC_UpperCalcasieu',
 'deliverables/20221222_08080203_UC_RAS2DModelSetupP1/UC_UpperCalcasieu',
 'deliverables/20221222_08080203_UC_RAS2DModelSetupP1/UC_UpperCalcasieu',
 'deliverables/20221222_08080205_WF_RAS2DModelSetupP1/WF_WestForkCalcasieu',
 'deliverables/20221222_08080205_WF_RAS2DModelSetupP1/WF_WestForkCalcasieu',
 'deliverables/20221222_08080205_WF_RAS2DModelSetupP1/WF_WestForkCalcasieu',
 'deliverables/20221222_08080205_WF_RAS2DModelSetupP1/WF_WestForkCalcasieu',
 'deliverables/20221222_08080205_WF_RAS2DModelSetupP1/WF_WestForkCalcasieu',
 'deliverables/20221222_08080206_LC_RAS2DModelSetupP1/LC_LowerCalcasieu',
 'deliverables/20221222_08080206_LC_RAS2DModelSetupP1/LC_LowerCalcasieu',
 'deliverables/20221222_08080206_LC_RAS2DModelSetupP1/LC_LowerCalcasieu',
 'deliverables/20230310_08080204_WH_RASModelSetup',
 'deliverables/20230310_08080204_WH_RASModelSetup',
 'deliverables/20230310_08080204_WH_RASModelSetup',
 'deliverables/

In [80]:
# Remove duplicates from list of model directories
for region in model_dirs:
    model_dirs[region]['dirs'] = list(set(model_dirs[region]['dirs']))
    print(region, len(model_dirs[region]['dirs']))

lwi-region4 12
lwi-region5 9
lwi-region6 5
lwi-region7 29


In [92]:
# for each model directory get first plan file.
last_plan_files = {}
for region in model_dirs:
    last_plan_files[region] = {}
    # print(region)
    for model_dir in model_dirs[region]['dirs']:
        last_plan_files[region][model_dir] = {}
        # print(model_dir)
        for plan_hdf in models[region]['ras']['plan_hdfs']:
            if model_dir in plan_hdf:
                last_plan_files[region][model_dir]['last_plan_file'] = plan_hdf
                # print(plan_hdf)
#                 model_dirs['first_plan_file'] = plan_hdf


# last_plan_files['lwi-region4']['deliverables/20230504_08080203_UC_RAS1D2DModelSetupP2/UC_UpperCalcasieu-2D']

{'last_plan_file': 'deliverables/20230504_08080203_UC_RAS1D2DModelSetupP2/UC_UpperCalcasieu-2D/UC_UpperCalcasieu.p02.hdf'}

In [93]:
# For each plan, open hdf and get model name, description, and ras type (1D, 2D, etc.), and pull a geospatial extent from the hdf.
for region in last_plan_files:
    for model_dir in last_plan_files[region]:
        for plan in last_plan_files[region][model_dir]:
            print(last_plan_files[region][model_dir][plan])
            with h5py.File(last_plan_files[region][model_dir][plan], "r") as f:
                print (f.keys())


deliverables/20230504_08080203_UC_RAS1D2DModelSetupP2/UC_UpperCalcasieu-2D/UC_UpperCalcasieu.p02.hdf
deliverables/20230606_08080206_LC_RAS2DModelSetupP2/LC_LowerCalcasieu.p08.hdf
deliverables/20221222_08080203_UC_RAS2DModelSetupP1/UC_UpperCalcasieu/UC_UpperCalcasieu.p40.hdf
deliverables/20230606_08080205_WF_RAS1D2DModelSetupP2/WF_WestForkCalcasieu-1D/WF_WestForkCalcasieu.p11.hdf
deliverables/20221222_08080205_WF_RAS2DModelSetupP1/WF_WestForkCalcasieu/WestForkCalcasieu_2.p24.hdf
deliverables/20230504_08080203_UC_RAS1D2DModelSetupP2/UC_UpperCalcasieu-1D/UC_Calcasieu.p02.hdf
deliverables/20230321_12010005_SL_RASModelSetup/LowerSabine.p02.hdf
deliverables/20230606_08080205_WF_RAS1D2DModelSetupP2/04_BoundaryConditions/Stage/Region4Stage/WF_WestForkCalcasieu.p01.hdf
deliverables/20221222_08080206_LC_RAS2DModelSetupP1/LC_LowerCalcasieu/LC_LowerCalcasieu.p08.hdf
deliverables/20230310_12010004_TO_RASModelSetup/TO_ToledoBend.p03.hdf
deliverables/20230310_08080204_WH_RASModelSetup/WH_Whisky_Chitt